In [1]:
import nrrd

In [2]:
path = 'data/images/LIDC-IDRI-0001.nrrd'

V,d = nrrd.read(path)

In [3]:
V.shape

(512, 512, 133)

In [4]:
d

OrderedDict([('type', 'int16'),
             ('dimension', 3),
             ('sizes', array([512, 512, 133])),
             ('endian', 'little'),
             ('encoding', 'gzip'),
             ('patient_id', 'LIDC-IDRI-0001'),
             ('slice_thickness', '2.5'),
             ('pixel_spacing', '0.703125'),
             ('contrast_used', 'True')])

In [5]:
import code.ImageSliceViewer3D as isv
from scipy.ndimage import rotate   # for visualization purposes 

im3D = rotate(V[0], 270)

ModuleNotFoundError: No module named 'code.ImageSliceViewer3D'; 'code' is not a package

In [ ]:
isv.ImageSliceViewer3D(im3D)